In [ ]:
# Install needed stuff to the Jupyter and kernel environment
# Need to install only once, so commented out for cleaner output
# %pip install pyarrow
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install pyfinance
# %pip install yfinance
# %pip install yahoo-fin

In [ ]:
# Import needed stuff
import sys
import pyarrow.feather as feather
import math
import pandas
import numpy as np
import matplotlib.pyplot as plt
from pyfinance.options import BSM
import yfinance as yf
from datetime import datetime, timedelta
from yahoo_fin import options as op

In [ ]:
# General Delta-hedging function
def number_of_shares_to_buy(shares_per_contract, number_of_options, delta_of_option):
    delta_of_underlying = 1 # why?

    # Delta-hedging says:
    # number_of_shares * delta_of_underlying + number_of_options * delta_of_option = 0
    number_of_shares = (-number_of_options * delta_of_option / delta_of_underlying ) * shares_per_contract

    print("To delta-hedge the portoflio, you need to",("buy" if number_of_shares >= 0 else "sell"), round(np.abs(number_of_shares),2), "shares.")
    return number_of_shares


In [ ]:
# General volatility function
def get_volatility(stock_data):
    log_returns = np.log(stock_data/stock_data.shift(1)).dropna()
    volatility = log_returns.rolling(window = 21).std() * np.sqrt(252)
    return volatility
    

In [ ]:
# General plot function
def plot_stock_price_and_volatility(stock_prices, volatility):
    fig, ax = plt.subplots()
    ax.plot(stock_prices, color='red')
    ax.set_xlabel('Date', fontsize = 14)
    ax.set_ylabel('Underlying price', color='red', fontsize = 14)
    
    ax2 = ax.twinx()
    ax2.plot(volatility, color = 'blue')
    ax2.set_ylabel('Volatility', color = 'blue', fontsize = 14)
    
    plt.show()
    

## Let's use Apple stock data from Refinitiv here

In [ ]:
# Read data and show it, data is from Refinitiv workspace
apple_stock_data_as_df = feather.read_feather('../data/fe2.feather')

# Set date to index instead of running index from 1..n
date_as_index = apple_stock_data_as_df.set_index('Date', inplace=False)

vola = get_volatility(date_as_index['Underlying'])

# Set values
shares_per_contract = 100 # How this is determined ?

number_of_options = 1 # How this is determined? 1 stock = 1 option, n stocks = n options?

# Use Black-Scholes model to compute the delta of the call option
# T = maturity and r = interest rate just thrown there, no spesific reason I guess to be those
call_delta = BSM(kind='call', S0=date_as_index['Underlying'].iloc[-1], K=date_as_index['Underlying'].iloc[-1], T=0.25, r=0.05, sigma=vola.iloc[-1]).delta()

# Delta of the call option is always positive and between 0 and 1
print("Delta of the call option:", call_delta)

# Use Black-Scholes model to compute the delta of the put option
# Again T = maturity and r = interest rate are just thrown there, no spesific reason to be those (I think)
put_delta = BSM(kind='put', S0=date_as_index['Underlying'].iloc[-1], K=date_as_index['Underlying'].iloc[-1], T=0.25, r=0.05, sigma=vola.iloc[-1]).delta()

# Delta of the put option is always negative and between 0 and -1
print("Delta of the put option:", put_delta)

# Get the amounts to balance, aka delta-hedge the portfolio
how_many_shares_to_sell = number_of_shares_to_buy(shares_per_contract, number_of_options, call_delta)

how_many_shares_to_buy = number_of_shares_to_buy(shares_per_contract, number_of_options, put_delta)

plot_stock_price_and_volatility(date_as_index['Underlying'], vola)

# Lets try with S&P 500 data

In [ ]:
stock_name = 'SPY'
years = 1
stock_data = yf.download(tickers = stock_name, start = datetime.today() - timedelta(days=years*365), end = datetime.today())

stock_data # Date as index, Open, High, Low, Close, Adj Close, Volume

stock_prices = stock_data['Adj Close'] # Closing prices
spy_vola = get_volatility(stock_prices)
plot_stock_price_and_volatility(stock_prices, spy_vola)

# Set values
shares_per_contract = 100 # How this is determined ?

number_of_options = 1 # How this is determined? 1 stock = 1 option, n stocks = n options?

# Use Black-Scholes model to compute the delta of the call option
# T = maturity and r = interest rate just thrown there, no spesific reason I guess to be those
call_delta = BSM(kind='call', S0=stock_prices.iloc[-1], K=stock_prices.iloc[-1], T=0.25, r=0.05, sigma=spy_vola.iloc[-1]).delta()

# Delta of the call option is always positive and between 0 and 1
print("Delta of the call option:", call_delta)

# Use Black-Scholes model to compute the delta of the put option
# Again T = maturity and r = interest rate are just thrown there, no spesific reason to be those (I think)
put_delta = BSM(kind='put', S0=stock_prices.iloc[-1], K=stock_prices.iloc[-1], T=0.25, r=0.05, sigma=spy_vola.iloc[-1]).delta()

# Delta of the put option is always negative and between 0 and -1
print("Delta of the put option:", put_delta)

# Get the amounts to balance, aka delta-hedge the portfolio
how_many_shares_to_sell = number_of_shares_to_buy(shares_per_contract, number_of_options, call_delta)

how_many_shares_to_buy = number_of_shares_to_buy(shares_per_contract, number_of_options, put_delta)

## Hedging assigment

In [ ]:
print(stock_data) # our data

# days_to_maturity = 45/365 # 45 days to maturity ?
# t0 =  datetime.now() - timedelta(days = 45)
# t0 = stock_data.index() # time at t = 0, this takes the first date of the first value of the dataframe
# c0 = stock_data['Adj Close'].iloc[0] # call option with price C0 at time t0
# i0 = BSM(kind='call', S0=c0, K=c0, T=days_to_maturity, r=0.05, sigma=0.5).implied_vol(2.5) # implied volatility at t=0 to get delta0

# print(t0)
# print(c0)
# print(i0)

# Construct two portfolios
# op = long position in one call, op0 = c0
# re = short position containing delta amount of the underlying asset S, re0 = delta0 * s0 (stock price at 0)
# Changes in the protfolio OP are neutralized by oppsite changes in the replicatin portfolio re
# rehedge every second day
# Compute how much every portfolio has changed
# Count mean squared error


# Fetch real option data using Yahoo_fin

In [ ]:
ticker = 'SPY'
expiration_dates = op.get_expiration_dates(ticker)
exp = pandas.DataFrame(expiration_dates)
# print(exp)
calls = op.get_calls(ticker, date = expiration_dates[14]) # Chose 14 
c = pandas.DataFrame(calls)
print(c)

long_position_in_call = c.iloc[0]
print(long_position_in_call)
print(long_position_in_call['Strike'])


#puts = op.get_puts(ticker, date = expiration_dates[14])
#print(puts)
#chain_data = op.get_options_chain(ticker, date=expiration_dates[14])
#print(chain_data)


# Fetch using yfinance

In [ ]:
sp500 = yf.Ticker(ticker)
expiration_dates = pandas.DataFrame(data=sp500.options)
#print(expiration_dates)

option_chain_for_expiration = sp500.option_chain(expiration_dates.iloc[13][0]) # expiration 29.12.2023
#print(option_chain_for_expiration)

print('============== CALLS ==============')

opt_calls = option_chain_for_expiration.calls
print(opt_calls)

# print('============== PUTS ==============')

# opt_puts = option_chain_for_expiration.puts
# print(opt_puts)